In [1]:
###############################################
#Chapitre : Importer les vtk et les visualiser#
###############################################

import sys
import gdcm
import dicom
import os
import vtkgdcm
import numpy
import vtk
import itk
sys.path.append('/usr/lib/python27/dist-packages/')
import vtkDICOMPython

from vtk.util.misc import vtkGetDataRoot

In [2]:
#Créer la visualisation


# Create a rendering window and renderer
ren = vtk.vtkRenderer()
ren.SetBackground(0.5,0.5,0.5)
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(1024, 1024)

# Create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Enable user interface interactor
iren.Initialize()
renWin.Render()

In [3]:
# Lire les DICOM images (version VTK)  NB: par défaut, critère du image number
PathDicom2="/home/jonathan/Bureau/DICOMcochon/UH/"

directory2 = vtkDICOMPython.vtkDICOMDirectory()
directory2.SetDirectoryName(PathDicom2)
directory2.Update()

readerImg = vtkDICOMPython.vtkDICOMReader()
readerImg.SetFileNames(directory2.GetFileNamesForSeries(0))
readerImg.Update()

dim = readerImg.GetOutput().GetDimensions()

# Lire les DICOM images par coupes

actorImg1 = vtk.vtkImageActor()#Afficher la 1ère coupe au choix
actorImg1.SetInputData(readerImg.GetOutput())
actorImg2 = vtk.vtkImageActor()#Afficher la 2ème coupe au choix
actorImg2.SetInputData(readerImg.GetOutput())
actorImg3 = vtk.vtkImageActor()#Afficher la 3ème coupe au choix
actorImg3.SetInputData(readerImg.GetOutput())

mapperImg1 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg1.GetMapper())
mapperImg2 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg2.GetMapper())
mapperImg3 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg3.GetMapper())
actorImg1.GetMapper()
actorImg2.GetMapper()
actorImg3.GetMapper()
mapperImg1.SetSliceNumber(0)
mapperImg2.SetSliceNumber(dim[2]/5)
mapperImg3.SetSliceNumber(dim[2]/3)

ren.AddActor(actorImg1)
ren.AddActor(actorImg2)
ren.AddActor(actorImg3)

ren.ResetCamera()
renWin.Render()

In [4]:
#Marching Cubes DICOM Images

ImgMC = vtk.vtkMarchingCubes()
ImgMC.SetInputConnection(readerImg.GetOutputPort())
ImgMC.GenerateValues(1,150,300)
ImgMC.Update()

dim = readerImg.GetOutput().GetDimensions()

#Remplacer les polyDataNormals d'entrées avec les PolyData transformées
NormalsMC=vtk.vtkPolyDataNormals()
NormalsMC.SetInputConnection(ImgMC.GetOutputPort())
#Normals.SetFeatureAngle(60)
NormalsMC.Update()

stripperMC=vtk.vtkStripper()
stripperMC.SetInputConnection(NormalsMC.GetOutputPort())
stripperMC.Update()

mapperMC = vtk.vtkPolyDataMapper()
mapperMC.SetInputConnection(stripperMC.GetOutputPort())
mapperMC.ScalarVisibilityOff()
actorMC = vtk.vtkActor()
actorMC.SetMapper(mapperMC)
actorMC.GetProperty().SetColor(0.7,1,0.5)

ren.AddActor(actorMC)

ren.ResetCamera()
renWin.Render()

In [5]:
#RT Structure 2

#2: HolmiumG
filenameHoG="/home/jonathan/Bureau/DICOMcochon/vtk/Francisco.vtk"
RTStrucHoG = vtk.vtkPolyDataReader()
RTStrucHoG.SetFileName(filenameHoG)
RTStrucHoG.Update()

#2: HoG
mapperHoG = vtk.vtkPolyDataMapper()
#mapperHoG.SetInputConnection(TransfFiltreHoG.GetOutputPort())
mapperHoG.SetInputConnection(RTStrucHoG.GetOutputPort())
mapperHoG.ScalarVisibilityOff()
actorHoG = vtk.vtkActor()
actorHoG.SetMapper(mapperHoG)
actorHoG.GetProperty().SetColor(1,0,1)

#ren.AddActor(actorHoG)

ren.ResetCamera()
renWin.Render()


In [6]:
#Utiliser la matrice des DICOM images pour les lier avec les vtk
Transf1=vtk.vtkTransform()
Transf1.Scale(-1.,-1.,1.)
Transf1.Update()

TransfFiltre1=vtk.vtkTransformPolyDataFilter()
TransfFiltre1.SetInputConnection(RTStrucHoG.GetOutputPort())
TransfFiltre1.SetTransform(Transf1)
TransfFiltre1.Update()

Transf=vtk.vtkTransform()
Transf.SetMatrix(readerImg.GetPatientMatrix())
Transf.Inverse()
Transf.Update()

TransfFiltre=vtk.vtkTransformPolyDataFilter()
TransfFiltre.SetInputConnection(TransfFiltre1.GetOutputPort())
TransfFiltre.SetTransform(Transf)
TransfFiltre.Update()


Transf2=vtk.vtkTransform()
Transf2.Scale(1.,1.,-1.)
Transf2.Update()

TransfFiltre2=vtk.vtkTransformPolyDataFilter()
TransfFiltre2.SetInputConnection(TransfFiltre.GetOutputPort())
TransfFiltre2.SetTransform(Transf2)
TransfFiltre2.Update()




NormalsMC=vtk.vtkPolyDataNormals()
NormalsMC.SetInputConnection(TransfFiltre2.GetOutputPort())
#Normals.SetFeatureAngle(60)
NormalsMC.Update()

#2: HoG
mapperHoG = vtk.vtkPolyDataMapper()
mapperHoG.SetInputConnection(NormalsMC.GetOutputPort())
#mapperHoG.SetInputConnection(RTStrucHoG.GetOutputPort())
mapperHoG.ScalarVisibilityOff()
actorHoG1 = vtk.vtkActor()
actorHoG1.SetMapper(mapperHoG)
actorHoG1.GetProperty().SetColor(0,1,1)

ren.AddActor(actorHoG1)

ren.ResetCamera()
renWin.Render()

In [ ]:
ren.GetActiveCamera().Yaw(10)
ren.ResetCamera()
renWin.Render()

In [7]:

iren.Start()
